In [2]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import java.sql.Date

val weather = Seq(
  // --- Données du 10 janvier ---
  ("12345", Date.valueOf("2024-01-10"), 0,     5.0, 1.0),   // 00h00
  ("12345", Date.valueOf("2024-01-10"), 300,   6.0, 2.0),   // 03h00
  ("12345", Date.valueOf("2024-01-10"), 2300,  7.0, 3.0),   // 23h00

  // --- Données du 11 janvier ---
  ("12345", Date.valueOf("2024-01-11"), 0,     8.0, 4.0),   // 00h00
  ("12345", Date.valueOf("2024-01-11"), 1200,  9.0, 5.0)    // 12h00
).toDF("WBAN", "WDATE", "WTIME_HHMM", "TEMP", "WIND")

weather = [WBAN: string, WDATE: date ... 3 more fields]


[WBAN: string, WDATE: date ... 3 more fields]

In [4]:
weather.show(10)

+-----+----------+----------+----+----+
| WBAN|     WDATE|WTIME_HHMM|TEMP|WIND|
+-----+----------+----------+----+----+
|12345|2024-01-10|         0| 5.0| 1.0|
|12345|2024-01-10|       300| 6.0| 2.0|
|12345|2024-01-10|      2300| 7.0| 3.0|
|12345|2024-01-11|         0| 8.0| 4.0|
|12345|2024-01-11|      1200| 9.0| 5.0|
+-----+----------+----------+----+----+



In [8]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.Column

def hhmmHourCol(c: Column): Column = {
    val s  = regexp_replace(c.cast("string"), ":", "")
    val p4 = lpad(s, 4, "0")
    (substring(p4, 1, 2).cast("int") % 24)
}

/**
  * Prépare les données météo pour la jointure avec les vols.
  *
  * Idée :
  *  - Pour chaque enregistrement météo (WBAN, WDATE, WTIME_HHMM),
  *    on crée deux vues :
  *      1) météo du jour :      DATE = WDATE,  relHour = hour ∈ [0, 23]
  *      2) météo de la veille : DATE = WDATE + 1, relHour = hour - 24 ∈ [-24, -1]
  *
  *  Ainsi, pour un vol à la date D, on peut récupérer :
  *    - la météo du jour D       (relHour >= 0)
  *    - la météo de la veille D-1 (relHour < 0)
  */
def buildWeatherWithRelativeHour(weather: DataFrame): DataFrame = {

  // 1) Normalisation de l'heure (WTIME_HHMM → hour ∈ [0, 23])
  val weatherWithHour = weather
    .withColumn("hour", hhmmHourCol(col("WTIME_HHMM")))
    // on élimine les lignes où l'heure est invalide (NULL ou hors [0,23])
    .filter(col("hour").between(0, 23))

  // 2) Météo du jour même : DATE = WDATE, relHour = hour
  val sameDateWeather = weatherWithHour
    .withColumn("DATE", col("WDATE"))
    .withColumn("relHour", col("hour"))

  // 3) Météo de la veille, rattachée au jour suivant :
  //    Exemple : WDATE = 10/01 23h → DATE = 11/01, relHour = -1
  val previousDayWeatherForNextDate = weatherWithHour
    .withColumn("DATE", date_add(col("WDATE"), 1))
    .withColumn("relHour", col("hour") - 24)

  // 4) Union des deux vues + garde seulement la fenêtre [-24, 23]
  val weatherWithRelativeHour = sameDateWeather
    .unionByName(previousDayWeatherForNextDate)
    .filter(col("relHour").between(-24, 23))

  weatherWithRelativeHour
}

val weatherWithRelatievHours = buildWeatherWithRelativeHour(weather)
weatherWithRelatievHours.show(50)

weatherWithRelatievHours = [WBAN: string, WDATE: date ... 6 more fields]


hhmmHourCol: (c: org.apache.spark.sql.Column)org.apache.spark.sql.Column
buildWeatherWithRelativeHour: (weather: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


+-----+----------+----------+----+----+----+----------+-------+
| WBAN|     WDATE|WTIME_HHMM|TEMP|WIND|hour|      DATE|relHour|
+-----+----------+----------+----+----+----+----------+-------+
|12345|2024-01-10|         0| 5.0| 1.0|   0|2024-01-10|      0|
|12345|2024-01-10|       300| 6.0| 2.0|   3|2024-01-10|      3|
|12345|2024-01-10|      2300| 7.0| 3.0|  23|2024-01-10|     23|
|12345|2024-01-11|         0| 8.0| 4.0|   0|2024-01-11|      0|
|12345|2024-01-11|      1200| 9.0| 5.0|  12|2024-01-11|     12|
|12345|2024-01-10|         0| 5.0| 1.0|   0|2024-01-11|    -24|
|12345|2024-01-10|       300| 6.0| 2.0|   3|2024-01-11|    -21|
|12345|2024-01-10|      2300| 7.0| 3.0|  23|2024-01-11|     -1|
|12345|2024-01-11|         0| 8.0| 4.0|   0|2024-01-12|    -24|
|12345|2024-01-11|      1200| 9.0| 5.0|  12|2024-01-12|    -12|
+-----+----------+----------+----+----+----+----------+-------+



[WBAN: string, WDATE: date ... 6 more fields]